# 2. DBSCAN으로 학습하기 (star)

In [1]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

In [2]:
# data load
iris=pd.DataFrame(['data','target'])
iris.data = pd.read_csv("dataset/Stars.csv")
iris.target = pd.DataFrame(iris.data.Type)
iris.data = iris.data.drop(['Color', 'Spectral_Class', 'Type'], axis=1)
iris.data.columns=[0,1,2,3]
iris.target.columns = [0]

<ipython-input-2-5d83e6dcfeb2>:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  iris.data = pd.read_csv("dataset/Stars.csv")
<ipython-input-2-5d83e6dcfeb2>:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  iris.target = pd.DataFrame(iris.data.Type)


In [3]:
# Specify the eps value
from sklearn.cluster import DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

'''
eps의 값은 항상 positive 해야 하므로 for문에 넣을 수 없다.
따라서 계획 했던대로 eps의 값을 증가 시켜보며 학습하는 것은 힘들고
minpts의 값을 증가시키면서 학습해보겠다.
'''


'\neps의 값은 항상 positive 해야 하므로 for문에 넣을 수 없다.\n따라서 계획 했던대로 eps의 값을 증가 시켜보며 학습하는 것은 힘들고\nminpts의 값을 증가시키면서 학습해보겠다.\n'

In [5]:
# class count graph with minpts change
# min_samples는 1~100까지 돌려본다.

class_count=[0,]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result = dbscan.fit_predict(iris.data)
    class_count.append(len(np.unique(result)))

class_count = pd.DataFrame(class_count)
class_count = class_count[class_count[0]==6]
print(class_count)

Empty DataFrame
Columns: [0]
Index: []


In [6]:
# normal_iris_dbscan accuracy
# class의 개수가 데이터의 개수와 맞는 것의 accuracy를 측정해본다.

from sklearn.metrics.cluster import normalized_mutual_info_score

for i in class_count.index:
    dbscan = DBSCAN(min_samples=i)
    result = dbscan.fit_predict(iris.data)
    print('minpts=',i,":",normalized_mutual_info_score(iris.target[0], result))

In [9]:
# Scaler Preprocessed DBSCAN Accuracy

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

data_minmax = MinMaxScaler().fit_transform(iris.data)
data_maxabs = MaxAbsScaler().fit_transform(iris.data)
data_standard = StandardScaler().fit_transform(iris.data)
data_robust = RobustScaler().fit_transform(iris.data)
data_normalize = Normalizer().fit_transform(iris.data)
data_log = np.log1p(iris.data)

class_count_minmax, class_count_maxabs, class_count_standard, class_count_robust, class_count_normalize, class_count_log = [],[],[],[],[],[]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result_minmax = dbscan.fit_predict(data_minmax)
    result_maxabs = dbscan.fit_predict(data_maxabs)
    result_standard = dbscan.fit_predict(data_standard)
    result_robust = dbscan.fit_predict(data_robust)
    result_normalize = dbscan.fit_predict(data_normalize)

    class_count_minmax.append(len(np.unique(result_minmax)))
    class_count_maxabs.append(len(np.unique(result_maxabs)))
    class_count_standard.append(len(np.unique(result_standard)))
    class_count_robust.append(len(np.unique(result_robust)))
    class_count_normalize.append(len(np.unique(result_normalize)))

class_count_minmax = pd.DataFrame(class_count_minmax)
class_count_minmax = class_count_minmax[class_count_minmax[0]==6]
class_count_maxabs = pd.DataFrame(class_count_maxabs)
class_count_maxabs = class_count_maxabs[class_count_maxabs[0]==6]
class_count_standard = pd.DataFrame(class_count_standard)
class_count_standard = class_count_standard[class_count_standard[0]==6]
class_count_robust = pd.DataFrame(class_count_robust)
class_count_robust = class_count_robust[class_count_robust[0]==6]
class_count_normalize = pd.DataFrame(class_count_normalize)
class_count_normalize = class_count_normalize[class_count_normalize[0]==6]


In [10]:
score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []

def minpts_score(arr, data, score_list):
    for i in arr.index:
        dbscan = DBSCAN(min_samples=i)
        result = dbscan.fit_predict(data)
        score_list.append(normalized_mutual_info_score(iris.target[0], result))

minpts_score(class_count_minmax, data_minmax, score_minmax)
minpts_score(class_count_maxabs, data_maxabs, score_maxabs)
minpts_score(class_count_standard, data_standard, score_standard)
minpts_score(class_count_robust, data_robust, score_robust)
minpts_score(class_count_normalize, data_normalize, score_normalize)


In [11]:
def not_value(string, value):
    try:
        print(string, max(value))
    except ValueError:
        print(string, 0)

not_value('minmax score:', score_minmax)
not_value('maxabs score:', score_maxabs)
not_value('standard score:', score_standard)
not_value('robust score:', score_robust)
not_value('normalize score:', score_normalize)

minmax score: 0
maxabs score: 0
standard score: 0.5261509730414484
robust score: 0
normalize score: 0


In [14]:
# apply PCA
from sklearn.decomposition import PCA

model = PCA(n_components=2)

data_minmax = MinMaxScaler().fit_transform(iris.data)
data_maxabs = MaxAbsScaler().fit_transform(iris.data)
data_standard = StandardScaler().fit_transform(iris.data)
data_robust = RobustScaler().fit_transform(iris.data)
data_normalize = Normalizer().fit_transform(iris.data)

data_minmax = model.fit_transform(data_minmax)
data_maxabs = model.fit_transform(data_maxabs)
data_standard = model.fit_transform(data_standard)
data_robust = model.fit_transform(data_robust)
data_normalize = model.fit_transform(data_normalize)

class_count_minmax, class_count_maxabs, class_count_standard, class_count_robust, class_count_normalize, class_count_log = [],[],[],[],[],[]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result_minmax = dbscan.fit_predict(data_minmax)
    result_maxabs = dbscan.fit_predict(data_maxabs)
    result_standard = dbscan.fit_predict(data_standard)
    result_robust = dbscan.fit_predict(data_robust)
    result_normalize = dbscan.fit_predict(data_normalize)

    class_count_minmax.append(len(np.unique(result_minmax)))
    class_count_maxabs.append(len(np.unique(result_maxabs)))
    class_count_standard.append(len(np.unique(result_standard)))
    class_count_robust.append(len(np.unique(result_robust)))
    class_count_normalize.append(len(np.unique(result_normalize)))

class_count_minmax = pd.DataFrame(class_count_minmax)
class_count_minmax = class_count_minmax[class_count_minmax[0]==6]
class_count_maxabs = pd.DataFrame(class_count_maxabs)
class_count_maxabs = class_count_maxabs[class_count_maxabs[0]==6]
class_count_standard = pd.DataFrame(class_count_standard)
class_count_standard = class_count_standard[class_count_standard[0]==6]
class_count_robust = pd.DataFrame(class_count_robust)
class_count_robust = class_count_robust[class_count_robust[0]==6]
class_count_normalize = pd.DataFrame(class_count_normalize)
class_count_normalize = class_count_normalize[class_count_normalize[0]==6]

In [15]:
score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []

def minpts_score(arr, data, score_list):
    for i in arr.index:
        dbscan = DBSCAN(min_samples=i)
        result = dbscan.fit_predict(data)
        score_list.append(normalized_mutual_info_score(iris.target[0], result))

minpts_score(class_count_minmax, data_minmax, score_minmax)
minpts_score(class_count_maxabs, data_maxabs, score_maxabs)
minpts_score(class_count_standard, data_standard, score_standard)
minpts_score(class_count_robust, data_robust, score_robust)
minpts_score(class_count_normalize, data_normalize, score_normalize)

In [16]:
def not_value(string, value):
    try:
        print(string, max(value))
    except ValueError:
        print(string, 0)

not_value('minmax score:', score_minmax)
not_value('maxabs score:', score_maxabs)
not_value('standard score:', score_standard)
not_value('robust score:', score_robust)
not_value('normalize score:', score_normalize)

minmax score: 0
maxabs score: 0
standard score: 0.32127898808148164
robust score: 0
normalize score: 0


In [17]:
# Apply TSNE
from sklearn.manifold import TSNE

model = TSNE(n_components=2, random_state=0)

data_minmax = MinMaxScaler().fit_transform(iris.data)
data_maxabs = MaxAbsScaler().fit_transform(iris.data)
data_standard = StandardScaler().fit_transform(iris.data)
data_robust = RobustScaler().fit_transform(iris.data)
data_normalize = Normalizer().fit_transform(iris.data)

data_minmax = model.fit_transform(data_minmax)
data_maxabs = model.fit_transform(data_maxabs)
data_standard = model.fit_transform(data_standard)
data_robust = model.fit_transform(data_robust)
data_normalize = model.fit_transform(data_normalize)

class_count_minmax, class_count_maxabs, class_count_standard, class_count_robust, class_count_normalize, class_count_log = [],[],[],[],[],[]

for i in range(1,100,1):
    dbscan = DBSCAN(min_samples=i)
    result_minmax = dbscan.fit_predict(data_minmax)
    result_maxabs = dbscan.fit_predict(data_maxabs)
    result_standard = dbscan.fit_predict(data_standard)
    result_robust = dbscan.fit_predict(data_robust)
    result_normalize = dbscan.fit_predict(data_normalize)

    class_count_minmax.append(len(np.unique(result_minmax)))
    class_count_maxabs.append(len(np.unique(result_maxabs)))
    class_count_standard.append(len(np.unique(result_standard)))
    class_count_robust.append(len(np.unique(result_robust)))
    class_count_normalize.append(len(np.unique(result_normalize)))

class_count_minmax = pd.DataFrame(class_count_minmax)
class_count_minmax = class_count_minmax[class_count_minmax[0]==3]
class_count_maxabs = pd.DataFrame(class_count_maxabs)
class_count_maxabs = class_count_maxabs[class_count_maxabs[0]==3]
class_count_standard = pd.DataFrame(class_count_standard)
class_count_standard = class_count_standard[class_count_standard[0]==3]
class_count_robust = pd.DataFrame(class_count_robust)
class_count_robust = class_count_robust[class_count_robust[0]==3]
class_count_normalize = pd.DataFrame(class_count_normalize)
class_count_normalize = class_count_normalize[class_count_normalize[0]==3]

In [18]:
score_minmax, score_maxabs, score_standard, score_robust, score_normalize, score_log = [], [], [], [], [], []

def minpts_score(arr, data, score_list):
    for i in arr.index:
        dbscan = DBSCAN(min_samples=i)
        result = dbscan.fit_predict(data)
        score_list.append(normalized_mutual_info_score(iris.target[0], result))

minpts_score(class_count_minmax, data_minmax, score_minmax)
minpts_score(class_count_maxabs, data_maxabs, score_maxabs)
minpts_score(class_count_standard, data_standard, score_standard)
minpts_score(class_count_robust, data_robust, score_robust)
minpts_score(class_count_normalize, data_normalize, score_normalize)

In [19]:
def not_value(string, value):
    try:
        print(string, max(value))
    except ValueError:
        print(string, 0)

not_value('minmax score:', score_minmax)
not_value('maxabs score:', score_maxabs)
not_value('standard score:', score_standard)
not_value('robust score:', score_robust)
not_value('normalize score:', score_normalize)

minmax score: 0.20099914628911666
maxabs score: 0.20636085448441382
standard score: 0
robust score: 0.3238048201925838
normalize score: 0.2057305838852248
